In [52]:
import matplotlib
import matplotlib.pyplot as plt
import os
import matplotlib.patches as patches
import numpy as np
import pickle as pkl
import cv2
from Dataloader import Dataloader
from src.Vision import Vision
from src.DepthTracker import DepthTracker
from tqdm import tqdm
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
dl = Dataloader('/home/nic/ms-work/dtplayground/bear_front')
bbox,_ = dl.getBbox(0)
# static camera
T = np.array([0,0,0])
R = np.eye(3)

dt = DepthTracker(dl.K)
deltaT = 1/30.0
print(dl.numOfFrames)

295


In [81]:
for i in tqdm(range(dl.bboxes.shape[0])):
    scanFrame = False
    if i==0:
        scanFrame = True
        xyz = dl.getXYZ(i)
    else:
        xyz = np.empty((480,640,3))
        xyz[:,:,:] = np.nan
    img = dl.getRGB(i)

    bbox,_ = dl.getBbox(i)
    bestParticle = dt.updateMeasurements(img,xyz,bbox,T,R,deltaT,scanFrame)
    if bestParticle is not None:
        gt_xyz = dl.getXYZ(i)
        u,v = dt.vision.getBBoxCenter(bbox[0])
        gt_origin = gt_xyz[u,v,:]
        # print(bestParticle[:3]-gt_origin)

1%|          | 3/281 [00:00<00:33,  8.39it/s][ 0.0410678  -0.09916365  0.24885539]
[ 0.0410678  -0.09805218  0.24885539]
  2%|▏         | 6/281 [00:00<00:29,  9.23it/s][ 0.03966457 -0.09762149  0.25685539]
[ 0.0394006  -0.0955097   0.25685539]
[ 0.0394006  -0.09524573  0.25685539]
  3%|▎         | 9/281 [00:00<00:28,  9.61it/s][ 0.0394006  -0.09339791  0.25685539]
[ 0.0394006  -0.09260599  0.25685539]
[ 0.04217927 -0.09221698  0.24885539]
  4%|▍         | 12/281 [00:01<00:26, 10.26it/s][ 0.04301287 -0.09221698  0.24885539]
[ 0.04301287 -0.09388418  0.24885539]
[ 0.04301287 -0.09471778  0.24885539]
  6%|▌         | 16/281 [00:01<00:21, 12.23it/s][ 0.04468007 -0.09833005  0.24885539]
[ 0.04468007 -0.09860791  0.24885539]
[ 0.04634727 -0.10027511  0.24885539]
[ 0.04718087 -0.10055298  0.24885539]
  7%|▋         | 20/281 [00:01<00:18, 14.24it/s][ 0.0473198  -0.10052519  0.25685539]
[ 0.05134887 -0.10194231  0.24885539]
[ 0.05412753 -0.10305378  0.24885539]
[ 0.05718407 -0.10555458  0.24885

In [56]:
xyz.shape

(480, 640, 3)